In [7]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
wine = load_wine()
X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import joblib, matplotlib.pyplot as plt, os
def train_evaluate_save(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'recall': recall_score(y_test, y_pred, average='macro'),
        'f1': f1_score(y_test, y_pred, average='macro')
    }
    os.makedirs('../models', exist_ok=True)
    joblib.dump(model, f'../models/{model_name}.pkl')
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=wine.target_names)
    disp.plot()
    plt.savefig(f'../results/{model_name}_confusion_matrix.png')
    plt.close()
    return metrics


In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr_metrics = train_evaluate_save(lr, 'logistic_regression')

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_metrics = train_evaluate_save(rf, 'random_forest')

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state=42)
svm_metrics = train_evaluate_save(svm, 'svm')


In [13]:
import pandas as pd
metrics_df = pd.DataFrame([lr_metrics, rf_metrics, svm_metrics], index=['Logistic Regression', 'Random Forest', 'SVM'])
metrics_df.to_csv('../results/model_comparison.csv')


In [15]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment("Wine Classification")
mlflow.set_tracking_uri("http://localhost:5000")

2025/09/18 12:38:45 INFO mlflow.tracking.fluent: Experiment with name 'Wine Classification' does not exist. Creating a new experiment.


In [19]:
def train_log_mlflow(model, model_name, params, experiment_name="Wine_Classification"):
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        metrics = train_evaluate_save(model, model_name)
        for metric, value in metrics.items():
            mlflow.log_metric(metric, value)
        mlflow.log_artifact(f'../results/{model_name}_confusion_matrix.png')
        mlflow.sklearn.log_model(model, "model")
        return mlflow.active_run().info.run_id


In [ ]:
lr_params1 = {'max_iter': 1000, 'C': 0.1}
lr_run1_id = train_log_mlflow(LogisticRegression(**lr_params1), 'lr_run1', lr_params1)  # Acc: 0.944
lr_params2 = {'max_iter': 1000, 'C': 1.0}
lr_run2_id = train_log_mlflow(LogisticRegression(**lr_params2), 'lr_run2', lr_params2)  # Acc: 1.0
lr_params3 = {'max_iter': 1000, 'C': 10.0}
lr_run3_id = train_log_mlflow(LogisticRegression(**lr_params3), 'lr_run3', lr_params3)  # Acc: 1.0


2025/09/18 12:40:59 INFO mlflow.tracking.fluent: Experiment with name 'Wine_Classification' does not exist. Creating a new experiment.
